# Part 1

In [61]:
# importing libraries

import requests # library to handle requests

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [39]:
# reading Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
toronto_df = pd.read_html(url)[0]

# removing all listings with "Not assigned"
toronto_df = toronto_df[~toronto_df['Borough'].isin(["Not assigned"])]

# joining data with the same postcode
sorted_df = toronto_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

sorted_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [40]:
sorted_df.shape

(103, 3)

# Part 2

In [7]:
# retrieving csv file and creating dataframe with latitude and longitude values
url = "http://cocl.us/Geospatial_data"
latlong_df = pd.read_csv(url)

In [13]:
# combining the toronto and latitude/longitude dataframes
concat_df = pd.concat([sorted_df, latlong_df[['Latitude', 'Longitude']]], axis = 1, sort = False)
concat_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Part 3

##### I'll be using the Etobicoke borough to find the most popular venues in neighbourhoods as well as clustering neighbourhood regions using folium

In [20]:
# creating a dataframe containing the Etobicoke borough

etobicoke_data = concat_df[concat_df['Borough'].str.contains('Etobicoke')].reset_index(drop=True)
etobicoke_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
3,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
4,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999


In [63]:
# Latitude and longitude of Etobicoke gained from Google
latitude = 43.6205
longitude = -79.513
print("The latitude of Etobicoke is: " + str(latitude) + "N and the longitude of Eboticoke is: "+ str(longitude) + "W")

The latitude of Etobicoke is: 43.6205N and the longitude of Eboticoke is: -79.513W


In [67]:
# create map of Etobicoke
map_etobicoke = folium.Map(location=[latitude, longitude], zoom_start=12)

# adds markers to map
for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etobicoke)  
    
map_etobicoke

In [68]:
# @ hidden_cell
CLIENT_ID = 'ZYBRRQAGX0JK5SML4ECLKT2ZLURR4UED5D3DJCDPT5LADUHU' # your Foursquare ID
CLIENT_SECRET = 'Y2T2WPXNLWCFQTRCV5GD4UQKNQXLRFGFOPVNC44ZQWATBRLO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: ZYBRRQAGX0JK5SML4ECLKT2ZLURR4UED5D3DJCDPT5LADUHU
CLIENT_SECRET:Y2T2WPXNLWCFQTRCV5GD4UQKNQXLRFGFOPVNC44ZQWATBRLO


In [69]:


limit = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [70]:
etobicoke_venues = getNearbyVenues(names=etobicoke_data['Neighbourhood'],
                                   latitudes=etobicoke_data['Latitude'],
                                   longitudes=etobicoke_data['Longitude']
                                  )

Humber Bay Shores, Mimico South, New Toronto
Alderwood, Long Branch
The Kingsway, Montgomery Road, Old Mill North
Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea
Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor
Islington Avenue
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Westmount
Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown
Northwest


In [71]:
# Shows size of dataframe as well as venues in the region
print(etobicoke_venues.shape)
etobicoke_venues.head()

(70, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Domino's Pizza,43.601676,-79.500908,Pizza Place
2,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
3,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
4,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Lucky Dice Restaurant,43.601392,-79.503056,Café


In [72]:
# Shows how many venues are in the neighbourhood regions
etobicoke_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",10,10,10,10,10,10
"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",9,9,9,9,9,9
"Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park",1,1,1,1,1,1
"Humber Bay Shores, Mimico South, New Toronto",12,12,12,12,12,12
"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",2,2,2,2,2,2
"Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips",3,3,3,3,3,3
"Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor",13,13,13,13,13,13
Northwest,2,2,2,2,2,2


In [73]:
# Check to see how many unique venues there are
print('There are {} uniques categories.'.format(len(etobicoke_venues['Venue Category'].unique())))

There are 37 uniques categories.


In [74]:
# one hot encoding
etobicoke_onehot = pd.get_dummies(etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etobicoke_onehot['Neighbourhood'] = etobicoke_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
etobicoke_onehot = etobicoke_onehot[fixed_columns]

etobicoke_onehot.head()

,Neighbourhood,American Restaurant,Athletics & Sports,Bakery,Bank,Baseball Field,Beer Store,Burger Joint,Burrito Place,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Discount Store,Drugstore,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Intersection,Liquor Store,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Supplement Shop,Thrift / Vintage Store,Wings Joint
0,"Humber Bay Shores, Mimico South, New Toronto",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Humber Bay Shores, Mimico South, New Toronto",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,"Humber Bay Shores, Mimico South, New Toronto",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,"Humber Bay Shores, Mimico South, New Toronto",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Humber Bay Shores, Mimico South, New Toronto",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
# Groups neighborhoods by frequency of venues in the area
etobicoke_grouped = etobicoke_onehot.groupby('Neighbourhood').mean().reset_index()
etobicoke_grouped

,Neighbourhood,American Restaurant,Athletics & Sports,Bakery,Bank,Baseball Field,Beer Store,Burger Joint,Burrito Place,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Discount Store,Drugstore,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Intersection,Liquor Store,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Supplement Shop,Thrift / Vintage Store,Wings Joint
0,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.0,0.0,0.111111,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.0,0.111111,0.111111,0.222222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.0,0.0,0.0,0.000000,0.0,0.111111,0.0,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.1,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.200000,0.1,0.1,0.0,0.000000,0.0,0.100000,0.1,0.000000,0.000000,0.000000
2,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.000000,0.0,0.000000,0.0,0.0,0.111111,0.000000,0.000000,0.111111,0.000000,0.111111,0.111111,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.111111,0.111111,0.111111,0.111111,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
3,"Cloverdale, Islington, Martin Grove, Princess ...",0.000000,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
4,"Humber Bay Shores, Mimico South, New Toronto",0.083333,0.0,0.083333,0.0,0.0,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.0,0.083333,0.083333,0.000000,0.083333,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.083333,0.083333,0.0,0.0,0.0,0.083333,0.0,0.083333,0.0,0.000000,0.000000,0.000000
5,"Humber Bay, King's Mill Park, Kingsway Park So...",0.000000,0.0,0.000000,0.0,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
6,"Kingsview Village, Martin Grove Gardens, Richv...",0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,0.333333,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
7,"Kingsway Park South West, Mimico NW, The Queen...",0.000000,0.0,0.076923,0.0,0.0,0.000000,0.076923,0.076923,0.000000,0.000000,0.000000,0.076923,0.076923,0.0,0.076923,0.000000,0.076923,0.076923,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.076923,0.0,0.076923,0.076923,0.076923
8,Northwest,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.5,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
9,"The Kingsway, Montgomery Road, Old Mill North",0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.5,0.000000,0.0,0.000000,0.000000,0.000000


In [76]:
# Shows top 5 venues in each neighbourhood region
num_top_venues = 5

for hood in etobicoke_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = etobicoke_grouped[etobicoke_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
            venue  freq
0   Grocery Store  0.22
1     Coffee Shop  0.11
2        Pharmacy  0.11
3      Beer Store  0.11
4  Sandwich Place  0.11


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place   0.2
1         Coffee Shop   0.1
2  Athletics & Sports   0.1
3                 Gym   0.1
4        Skating Rink   0.1


----Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.11
2         Park  0.11
3    Pet Store  0.11
4     Pharmacy  0.11


----Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park----
                       venue  freq
0                       Bank   1.0
1        American Restaurant   0.0
2                        Pub   0.0
3  Middle Eastern Restaurant   0.0
4          Mobile Phone Shop   0.0


----Humber Bay Shores, Mimico South, New Toron

In [77]:
# function that sorts venues 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
# creates dataframe that shows top 10 venues for each neighbourhood region
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = etobicoke_grouped['Neighbourhood']

for ind in np.arange(etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Beer Store,Fast Food Restaurant,Coffee Shop,Pharmacy,Fried Chicken Joint,Sandwich Place,Baseball Field,Burger Joint
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Athletics & Sports,Skating Rink,Sandwich Place,Gym,Pub,Pool,Coffee Shop,Burrito Place
2,"Bloordale Gardens, Eringate, Markland Wood, Ol...",Pizza Place,Pet Store,Convenience Store,Liquor Store,Beer Store,Coffee Shop,Café,Park,Pharmacy,Drugstore
3,"Cloverdale, Islington, Martin Grove, Princess ...",Bank,Wings Joint,Chinese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Coffee Shop,Café
4,"Humber Bay Shores, Mimico South, New Toronto",Café,American Restaurant,Pizza Place,Gym,Liquor Store,Fast Food Restaurant,Pharmacy,Restaurant,Sandwich Place,Bakery


Clustering neighbourhoods and producing a map from the clusters

In [79]:
# set number of clusters
kclusters = 5

etobicoke_grouped_clustering = etobicoke_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 0, 4, 0, 0, 2, 3], dtype=int32)

In [80]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

etobicoke_merged = etobicoke_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
etobicoke_merged = etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

etobicoke_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,0.0,Café,American Restaurant,Pizza Place,Gym,Liquor Store,Fast Food Restaurant,Pharmacy,Restaurant,Sandwich Place,Bakery
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,0.0,Pizza Place,Pharmacy,Athletics & Sports,Skating Rink,Sandwich Place,Gym,Pub,Pool,Coffee Shop,Burrito Place
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,3.0,River,Park,Wings Joint,Café,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Coffee Shop,Chinese Restaurant
3,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509,4.0,Pool,Baseball Field,Café,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store,Coffee Shop,Chinese Restaurant
4,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999,0.0,Wings Joint,Thrift / Vintage Store,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Grocery Store,Gym


In [90]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etobicoke_merged['Latitude'], etobicoke_merged['Longitude'], etobicoke_merged['Neighbourhood'], etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters